ASAP CRN Metadata validation 
# Team Scherzer. ASAP CRN Metadata validation 

10 Oct 2024
Andy Henrie




In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, create_upload_medadata_package

%load_ext autoreload
%autoreload 2


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


> SANITY CHECK: verify reading from google doc works.

```python
CDEv1_ = read_CDE("v1")
CDEv2_ = read_CDE("v2.1")
CDEv3_ = read_CDE("v3.0")
```


In [3]:
CDEv1_ = read_CDE("v1")
CDEv2_ = read_CDE("v2.1")
CDEv3_ = read_CDE("v3.0")

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
read url
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
read url
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
read url


## Clean V2 Table
write clean metadata tables according to CDE v2

### Team Scherzer

In [4]:
## convert 
team = "scherzer"
collection = "sn_pmdbs"

root_path = Path.home() / ("Projects/ASAP/data/teams")

metadata_path = root_path / f"{team}/{collection}/metadata/"
og_path = metadata_path / "og"



In [5]:
sheets = ["SAMPLE","SUBJECT","CLINPATH","STUDY","PROTOCOL","DATA"]

excel_path = og_path / "ASAP_CDE-ScherzerTeam.xlsx"
STUDY = pd.read_excel(excel_path,sheet_name="STUDY")#,header=1)#.drop(columns="Field")
CLINPATH = pd.read_excel(excel_path,sheet_name="CLINPATH")#,header=1)#.drop(columns="Field")
SUBJECT = pd.read_excel(excel_path,sheet_name="SUBJECT")#,header=1)#.drop(columns="Field")
SAMPLE = pd.read_excel(excel_path,sheet_name="SAMPLE")#,header=1)#.drop(columns="Field")
PROTOCOL = pd.read_excel(excel_path,sheet_name="PROTOCOL")#,header=1)#.drop(columns="Field")
DATA = pd.read_excel(excel_path,sheet_name="DATA")#,header=1)#.drop(columns="Field")

metadata_version = "v2"

METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

In [6]:
STUDY = prep_table(STUDY,CDEv2)
STUDY['metadata_version_date'] = METADATA_VERSION_DATE
STUDY.rename({'submittor_email':'submitter_email'}, axis=1, inplace=True)
STUDY['ASAP_team_name'] = "TEAM-SCHERZER"
STUDY['team_dataset_id'] = "MTG_snRNAseq" #"Scherzer_MTG_snRNAseq"


In [7]:
STUDY['alternate_dataset_id'] = NULL
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-SCHERZER,"Scherzer, Dong, and Levin",Parkinson5D: deconstructing proximal disease m...,MTG_snRNAseq,PD5D_MTG_snRNAseq,Here we will develop a molecular atlas of Park...,Dr.Clemens Scherzer,Clemens.scherzer@yale.edu,"Clemens, Scherzer; Xianjun, Dong; Joshua, Levi...",Dr.Xianjun Dong,...,Nan,94,MTG,PD and control postmortem brains,Nan,Nan,ASAP Access & Use Policy,Nan,v2_20241016,NA


In [8]:
# fix the excel saving source_subject_id as a date
SUBJECT = prep_table(SUBJECT,CDEv2)

recoder = {'2023-03-29 00:00:00': '03-29',
       '2023-04-15 00:00:00': '04-15', 
       '2023-06-02 00:00:00': '06-02',
        '2023-06-15 00:00:00': '06-15',
        '2023-07-04 00:00:00': '07-04', 
       '2023-10-22 00:00:00': '10-22',
       '2023-11-28 00:00:00': '11-28',
        '2023-12-04 00:00:00': '12-04',
       '2023-12-21 00:00:00': '12-21'}

SUBJECT['source_subject_id'] = SUBJECT['source_subject_id'].replace(recoder)

SUBJECT['source_subject_id'].unique()




array(['00-09', '03-29', '03-39', '03-41', '03-47', '03-48', '04-15',
       '04-52', '04-64', '05-36', '06-02', '06-15', '06-35', '06-44',
       '06-51', '06-55', '06-62', '07-04', '07-37', '07-46', '08-55',
       '09-34', '09-52', '10-22', '10-76', '11-28', '11-44', '11-60',
       '12-04', '12-06', '12-21', '12-66', '13-05', '13-08', '13-17',
       '13-39', '13-40', '13-51', '13-61', '14-12', '14-24', '14-30',
       '15-04', '15-06', '15-18', '15-35', '15-46', '15-54', '15-67',
       '15-78', '15-85', '16-14', '16-44', '17-19', '17-22', '17-26',
       '17-30', '17-37', '17-41', '17-47', '17-49', '17-52', '17-56',
       '17-62', '18-09', '18-12', '18-14', '18-05', '18-17', '18-22',
       '18-27', '18-36', '18-39', '18-42', '18-48', '18-49', '18-55',
       '18-62', '18-65', '18-72', '18-78', '19-02', '19-10', '19-34',
       '19-39', '19-57', '19-59', '19-60', '19-74', '20-03', '20-15',
       '99-44', '99-47', '99-66'], dtype=object)

In [9]:

SUBJECT['hx_melanoma'] = SUBJECT['hx_melanoma'].replace({'Present':'Yes', 'Unknown': NULL})
SUBJECT['cognitive_status'] = SUBJECT['cognitive_status'].replace({'Unknown': NULL })

# SUBJECT.replace("Unknown", NULL, inplace=True)

# eductation levels 
SUBJECT['education_level'] = SUBJECT['education_level'].replace({'Unknown': NULL })

# map "Unknown" to NULL

# map ['14', '12', '16', '18', '13', '8', '15', '20', '19', '17', '11', '22', '10', '21'] ???
SUBJECT

,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,BN0009,00-09,Nan,Nan,BSHRI,Human,Male,64,White,Unknown,...,Yes,NA,NA,Former smoker,30,33,Dementia,Unknown,Idiopathic PD,NaN
1,BN0329,03-29,Nan,Nan,BSHRI,Human,Male,79,White,Unknown,...,Yes,NA,14,Unknown,Unknown,23,Dementia,Unknown,Idiopathic PD,NaN
2,BN0339,03-39,Nan,Nan,BSHRI,Human,Male,86,White,Unknown,...,No,NA,12,Former smoker,50,33,Normal,Unknown,Healthy Control,NaN
3,BN0341,03-41,Nan,Nan,BSHRI,Human,Male,89,White,Unknown,...,No,NA,16,Never,0,34,Normal,Unknown,Healthy Control,NaN
4,BN0347,03-47,Nan,Nan,BSHRI,Human,Female,95,White,Unknown,...,No,NA,12,Former smoker,Unknown,33,MCI,Unknown,Healthy Control,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,BN2003,20-03,Nan,Nan,BSHRI,Human,Male,72,White,Unknown,...,Yes,NA,16,Former smoker,Unknown,33,Dementia,Unknown,Idiopathic PD,NaN
90,BN2015,20-15,Nan,Nan,BSHRI,Human,Male,90,White,Unknown,...,No,NA,20,Never,Unknown,34,Normal,Unknown,Prodromal motor PD,NaN
91,BN9944,99-44,Nan,Nan,BSHRI,Human,Male,69,White,Unknown,...,No,NA,15,Former smoker,Unknown,34,Normal,Unknown,Healthy Control,NaN
92,BN9947,99-47,Nan,Nan,BSHRI,Human,Male,84,White,Unknown,...,No,NA,20,Former smoker,Unknown,33,Normal,Unknown,Prodromal motor PD,NaN


In [10]:
SAMPLE = prep_table(SAMPLE,CDEv2)

SAMPLE['region_level_1'] = SAMPLE['region_level_1'].replace({'Temporal Lobe':'Temporal lobe'})
SAMPLE['molecular_source'] = SAMPLE['molecular_source'].replace({'PolyA_RNA':'PolyA RNA'})

In [11]:
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"
subj_sex_mapper = dict(zip(SUBJECT['subject_id'], SUBJECT['sex']))
SAMPLE['sex_ontology_term_id'] = SAMPLE['subject_id'].map(subj_sex_mapper).replace({"Male":"PATO:0000384 (male)", "Female":"PATO:0000383 (female)" })

In [12]:

# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

# autopsy coding is too too idosyncratic.  leave 
# the path_nia_aa_a are coded as path_nia_ri... swap them 

CLINPATH = prep_table(CLINPATH,CDEv2)


# 'Low', 'Intermediate', 'High', 'None'
CLINPATH['path_nia_ri'] = CLINPATH['path_nia_aa_a'].replace({'Low ADNC': 'Low', 
                                                            'Not AD': 'None', 
                                                            'Intermediate ADNC': 'Intermediate'})
CLINPATH['path_nia_aa_a'] = NULL


# path_thal
CLINPATH['path_thal']= CLINPATH['path_thal'].replace({'Phase 0 (A0)':'0', 
                                'Phase 1 (A1)':'1', 
                                'Phase 2 (A1)':'2', 
                                'Phase 3 (A2)':'3', 
                                'Phase 4 (A3)':'4',
                                'Phase 5 (A3)':'5', 
                                'Unknown':NULL, 
                                'Missing/unknown':NULL})

CLINPATH['path_infarcs'] = CLINPATH['path_infarcs'].replace('Unknown', NULL)


# no idea what No, undx, yes maps to.  leave
CLINPATH['TDP43'] = CLINPATH['TDP43'].replace('Unknown', NULL)

# sn_neuronal_loss has alternate 0-3 coding.  leave

# add 'replicate' coding (nans)

In [13]:

mckeith_map = {'l. Olfactory Bulb-Only':"Olfactory bulb only", 'lla. Brainstem Predominant':"Brainstem",
       'llb. Limbic Predominant':"Limbic (transitional)", 'lV. Neocortical':"Neocortical",
       'lll. Brainstem/Limbic':"Amygdala Predominant", '0. No Lewy bodies':"Absent", "Unknown":NULL}


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace(mckeith_map)

# leave path_nia_ri like this for now. not sure how to map "criteria not met" and "Not AD"

# leave amyloid_angiopathy_severity_scale like this for now. not sure how to map 'Cerebral amyloid angiopathy, temporal and occipital lobe','Cerebral amyloid angiopathy, frontal lobe']
CLINPATH

,subject_id,source_subject_id,duration_pmi,path_autopsy_dx_main,path_autopsy_second_dx,path_autopsy_third_dx,path_autopsy_fourth_dx,path_autopsy_fifth_dx,path_autopsy_sixth_dx,path_autopsy_seventh_dx,...,path_nia_ri,path_nia_aa_a,path_nia_aa_b,path_nia_aa_c,TDP43,arteriolosclerosis_severity_scale,amyloid_angiopathy_severity_scale,path_ad_level,dig_slide_avail,quant_path_avail
0,BN0009,00-09,4.00,PD/Dem,Charcot-Marie-Tooth disease (history),"GBA L444P/WT, L444P mutation",None,None,None,None,...,Nan,NA,Nan,Nan,NA,Nan,Nan,Unknown,Nan,Nan
1,BN0329,03-29,4.50,PD/Dem,Seizure disorder (history),None,None,None,None,None,...,Nan,NA,Nan,Nan,NA,Nan,Nan,Unknown,Nan,Nan
2,BN0339,03-39,2.75,Control,Non-diagnostic Alzheimer's changes,CAA,None,None,None,None,...,Nan,NA,Nan,Nan,NA,Mild,Moderate,Unknown,Nan,Nan
3,BN0341,03-41,2.50,Control,Non-diagnostic Alzheimer's changes,CWMR,None,None,None,None,...,Nan,NA,Nan,Nan,No,Moderate,Mild,Unknown,Nan,Nan
4,BN0347,03-47,3.50,Control (MCI),Non-diagnostic Alzheimer's changes,"Argyrophilic grains, mesial temporal lobe",Infarct(s),CWMR,Several microscopic foci of cerebellar cortica...,"Etat crible, putamen and caudate nucleus",...,Nan,NA,Nan,Nan,NA,Mild,Nan,Unknown,Nan,Nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,BN2003,20-03,2.65,PD/Dem,"Microscopic changes of Alzheimer's disease, in...","Focal non-specific glial tauopathy, cortex of ...",None,None,None,None,...,Intermediate,NA,Nan,Nan,No,Unknown,Unknown,Unknown,Nan,Nan
90,BN2015,20-15,5.40,Control (history),"Microscopic changes of Alzheimer's disease, in...",Incidental Lewy body disease,None,None,None,None,...,Nan,NA,Nan,Nan,No,Unknown,Unknown,Unknown,Nan,Nan
91,BN9944,99-44,2.16,Control,Non-diagnostic Alzheimer's changes,Alzheimer Type II astrocytosis consistent with...,None,None,None,None,...,Nan,NA,Nan,Nan,NA,Unknown,Nan,Unknown,Nan,Nan
92,BN9947,99-47,2.50,Control,Non-diagnostic Alzheimer's changes,Alzheimer type II astrocytosis,Inc LBs,None,None,None,...,Nan,NA,Nan,Nan,NA,Unknown,Nan,Unknown,Nan,Nan


Chat GPT help

```python

mapping = {
    'PD/Dem': ['Parkinson\'s disease with dementia', 'Dementia with Lewy bodies'],
    'Control': ['Control, no misfolded protein or significant vascular pathology'],
    'Control (MCI)': ['Control, Low level AD neuropathological change', 
                       'Control, Limbic predominant age-related TDP43 proteinopathy (LATE)'],
    'PD/ND': ['Parkinson\'s disease'],
    'Control, Non-motoric': ['Control, no misfolded protein or significant vascular pathology'],
    'PDD': ['Parkinson\'s disease with dementia', 'Dementia with Lewy bodies'],
    'MCI': ['Alzheimer\'s disease (intermediate level neuropathological change)', 
            'Control, Low level AD neuropathological change'],
    'PD (MCI)': ['Parkinson\'s disease with dementia', 'Parkinson\'s disease'],
    'Control (history)': ['Control, no misfolded protein or significant vascular pathology'],
    'Control (diseased)': ['Control, Cerebrovascular disease (atherosclerosis)', 
                           'Control, Cerebrovascular disease (hyaline arteriolosclerosis)', 
                           'Control, Cerebrovascular disease (cerebral amyloid angiopathy)', 
                           'Other neurological disorder'],
    'MCI (clinical history)': ['Control, Low level AD neuropathological change', 
                               'Control, Limbic predominant age-related TDP43 proteinopathy (LATE)'],
    'Control (clinical history)': ['Control, no misfolded protein or significant vascular pathology']
}


```

In [14]:
# path_autopsy_main_mapper = {
#     'PD/Dem': 'Parkinson\'s disease with dementia',
#     'Control': 'Control, no misfolded protein or significant vascular pathology',
#     'Control (MCI)': 'Control, Low level AD neuropathological change',
#     'PD/ND': 'Parkinson\'s disease',
#     'Control, Non-motoric': 'Control, no misfolded protein or significant vascular pathology',
#     'PDD': 'Parkinson\'s disease with dementia',
#     'MCI': 'Alzheimer\'s disease (intermediate level neuropathological change)',
#     'PD (MCI)': 'Parkinson\'s disease with dementia',
#     'Control (history)': 'Control, no misfolded protein or significant vascular pathology',
#     'Control (diseased)': 'Control, Cerebrovascular disease (atherosclerosis)',
#     'MCI (clinical history)': 'Control, Low level AD neuropathological change',
#     'Control (clinical history)': 'Control, no misfolded protein or significant vascular pathology'
# }

# CLINPATH['path_autopsy_dx_main'] = CLINPATH['path_autopsy_dx_main'].replace(path_autopsy_main_mapper)

In [15]:
v2_path = metadata_path / "v2"
STUDY.to_csv(v2_path / f"STUDY.csv", index=False)
SAMPLE.to_csv(v2_path / f"SAMPLE.csv", index=False)
SUBJECT.to_csv(v2_path / f"SUBJECT.csv", index=False)
PROTOCOL.to_csv(v2_path / f"PROTOCOL.csv", index=False)
CLINPATH.to_csv(v2_path / f"CLINPATH.csv", index=False)
DATA.to_csv(v2_path / f"DATA.csv", index=False)

In [16]:
CDE = CDEv2
tables = CDE['Table'].unique()

dfs = {}
for table in tables:
    df = read_meta_table(v2_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table
    # df.to_csv(v1_path / f"{table}.csv", index=False)

recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **7 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCHID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ **15 Fields with empty (NULL) values:**

	- AMPPD_id: 94/94 empty rows (REQUIRED)

	- GP2_id: 94/94 empty rows (REQUIRED)

	- ethnicity: 94/94 empty rows (REQUIRED)

	- family_history: 86/94 emp

In [17]:
# make tables conform to CDE and save extra columns as "auxiliary"
v2_path = metadata_path / "v2"

for table in tables:
    df = dfs[table]
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(v2_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v2_path / f"{table}.csv", index=False)

Saved SAMPLE_auxiliary.csv
Saved DATA_auxiliary.csv


validate v2 tables


validate v2 tables


In [18]:
CDE = CDEv2
v2_tables = dfs
for table,df in v2_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()


recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **7 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCHID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ **15 Fields with empty (NULL) values:**

	- AMPPD_id: 94/94 empty rows (REQUIRED)

	- GP2_id: 94/94 empty rows (REQUIRED)

	- ethnicity: 94/94 empty rows (REQUIRED)

	- family_history: 86/94 emp

In [19]:
v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

f"{v3_meta_tables}"

"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']"

In [20]:
v3_path = metadata_path / "v3"

v3_tables, aux_tables = v2_to_v3_PMDBS(v2_path, v3_path, CDEv2, CDEv3)

recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
recoding replicate_count as int
recoding repeated_sample as int


In [21]:
CDE = CDEv3
for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **7 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- primary_diagnosis_text: 94/94 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **13 Fields with empty (NULL) values:**

	- o

In [22]:
v3_tables['CONDITION']

Field,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,idiopathic_pd,Case-Control,Case,NaN,NaN
1,healthy_control,Case-Control,Control,NaN,NaN
2,prodromal_motor_pd,Case-Control,Other,NaN,NaN


In [23]:
STUDY = v3_tables['STUDY']
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-SCHERZER,"Scherzer, Dong, and Levin",Parkinson5D: deconstructing proximal disease m...,MTG_snRNAseq,PD5D_MTG_snRNAseq,Here we will develop a molecular atlas of Park...,Dr.Clemens Scherzer,Clemens.scherzer@yale.edu,"Clemens, Scherzer; Xianjun, Dong; Joshua, Levi...",Dr.Xianjun Dong,...,94,MTG,PD and control postmortem brains,ASAP Access & Use Policy,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",NA,NA,NA,v2_20241016,NA


### Create metadata package


In [24]:
export_path = root_path / f"{team}"

create_upload_medadata_package(export_path, v3_tables)

In [25]:
v3_tables['SAMPLE']['condition_id']

0          idiopathic_pd
1          idiopathic_pd
2        healthy_control
3        healthy_control
4        healthy_control
             ...        
92         idiopathic_pd
93    prodromal_motor_pd
94       healthy_control
95    prodromal_motor_pd
96    prodromal_motor_pd
Name: condition_id, Length: 97, dtype: object

In [26]:
v3_tables['CONDITION']

Field,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,idiopathic_pd,Case-Control,Case,NaN,NaN
1,healthy_control,Case-Control,Control,NaN,NaN
2,prodromal_motor_pd,Case-Control,Other,NaN,NaN


In [ ]:
import shutil
import os
from IPython.display import display, Javascript

def save_and_copy_notebook(copy_name):
    # Step 1: Save the notebook
    display(Javascript('IPython.notebook.save_checkpoint();'))
    
    # Step 2: Get the current notebook name
    notebook_path = os.path.abspath('')  # Get the current working directory
    notebook_name = [f for f in os.listdir(notebook_path) if f.endswith('.ipynb')]
    
    if len(notebook_name) > 0:
        current_notebook = notebook_name[0]  # This assumes there's only one notebook in the directory
        
        # Step 3: Create a copy of the notebook
        copy_path = os.path.join(notebook_path, copy_name)
        shutil.copy(current_notebook, copy_path)
        
        print(f'Notebook saved and copied as: {copy_name}')
    else:
        print("Couldn't find the notebook file in the directory.")

## Clean Tables
Some of the coding is wrong and easily fixed.  Some is not.  

TODO: share back details to Team Scherzer for better metadata

In [44]:
!pwd

/Users/ergonyc/Projects/ASAP/meta-clean


In [3]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/scherzer-credentials.json  



Activated service account credentials for: [raw-admin-scherzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



In [4]:

# !gcloud storage hash "gs://asap-raw-data-team-scherzer/**/*.gz"  --skip-crc32c --hex  --billing-project dnastack-asap-parkinsons > scherzer_hexhash.log

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-scherzer/**/*.gz" > scherzer_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [3]:
!gsutil -u dnastack-asap-parkinsons cp -r "/Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023" "gs://asap-raw-data-team-scherzer/metadata/"

Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/ASAP_CDE-ScherzerTeam.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/CLINPATH.csv [Content-Type=text/csv]...
Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/SUBJECT.csv [Content-Type=text/csv]...
Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/SAMPLE.csv [Content-Type=text/csv]...
- [4 files][540.2 KiB/540.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/DATA.csv [Content-T

In [4]:
# !gcloud storage ls -l "gs://asap-raw-data-team-scherzer/metadata"  --billing-project dnastack-asap-parkinsons 

     35161  2023-11-22T01:47:41Z  gs://asap-raw-data-team-scherzer/metadata/CLINPATH.csv
    595701  2023-11-22T01:47:41Z  gs://asap-raw-data-team-scherzer/metadata/DATA.csv
      1165  2023-11-22T01:47:40Z  gs://asap-raw-data-team-scherzer/metadata/PROTOCOL.csv
     32705  2023-11-22T01:47:40Z  gs://asap-raw-data-team-scherzer/metadata/SAMPLE.csv
      1684  2023-11-22T01:47:40Z  gs://asap-raw-data-team-scherzer/metadata/STUDY.csv
     15648  2023-11-22T01:47:40Z  gs://asap-raw-data-team-scherzer/metadata/SUBJECT.csv
                                  gs://asap-raw-data-team-scherzer/metadata/Scherzer_raw_meta11202023/
                                  gs://asap-raw-data-team-scherzer/metadata/v2/
TOTAL: 6 objects, 682064 bytes (666.08kiB)


## Team-Scherzer




In [45]:
## convert to seurat Object

## convert 
data_path = Path.home() / ("Projects/ASAP/team-scherzer")
metadata_path = data_path / "metadata"

metadata_version = "v2"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 
data_path = Path.home() / ("Projects/ASAP/team-scherzer")
metadata_path = data_path / "Scherzer_raw_meta11202023"

SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(metadata_path / "PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
DATA = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)



# # excel_path = data_path / "ASAP_CDE_team_Scherzer.v11152023.xlsx"
# excel_path = data_path / "Scherzer_raw_meta11202023/ASAP_CDE-ScherzerTeam.xlsx"
# STUDY = pd.read_excel(excel_path,sheet_name="STUDY")#,header=1)#.drop(columns="Field")
# CLINPATH = pd.read_excel(excel_path,sheet_name="CLINPATH")#,header=1)#.drop(columns="Field")
# SUBJECT = pd.read_excel(excel_path,sheet_name="SUBJECT")#,header=1)#.drop(columns="Field")
# SAMPLE = pd.read_excel(excel_path,sheet_name="SAMPLE")#,header=1)#.drop(columns="Field")
# PROTOCOL = pd.read_excel(excel_path,sheet_name="PROTOCOL")#,header=1)#.drop(columns="Field")
# DATA = pd.read_excel(excel_path,sheet_name="DATA")#,header=1)#.drop(columns="Field")


read url


In [77]:
CDE_df.to_json("ASAP_CDE_v2.json")

In [46]:
STUDY = prep_table(STUDY, CDE_df)


STUDY['metadata_version_date'] = METADATA_VERSION_DATE



In [47]:
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE_df)

report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [48]:
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE_df)
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()


All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [49]:
SUBJECT.head()

,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,BN0009,00-09,<NA>,<NA>,BSHRI,Human,Male,64,White,Unknown,...,yes,Unknown,unknown,Former smoker,30,33,Dementia,Unknown,Idiopathic PD,NaN
1,BN0329,03-29,<NA>,<NA>,BSHRI,Human,Male,79,White,Unknown,...,yes,Unknown,14,Unknown,Unknown,23,Dementia,Unknown,Idiopathic PD,NaN
2,BN0339,03-39,<NA>,<NA>,BSHRI,Human,Male,86,White,Unknown,...,no,Unknown,12,Former smoker,50,33,Normal,Unknown,Healthy Control,NaN
3,BN0341,03-41,<NA>,<NA>,BSHRI,Human,Male,89,White,Unknown,...,no,Unknown,16,Never,0,34,Normal,Unknown,Healthy Control,NaN
4,BN0347,03-47,<NA>,<NA>,BSHRI,Human,Female,95,White,Unknown,...,no,Unknown,12,Former smoker,Unknown,33,MCI,Unknown,Healthy Control,NaN


In [50]:
SUBJECT = prep_table(SUBJECT, CDE_df)

SUBJECT['hx_melanoma'] = SUBJECT['hx_melanoma'].replace({'Present':'Yes', 'Unknown': np.nan})


NameError: name 'np' is not defined

In [52]:
SUBJECT['cognitive_status'] = SUBJECT['cognitive_status'].replace({'Unknown': NULL  })

# SUBJECT.replace("Unknown", NULL, inplace=True)

# eductation levels 
SUBJECT['education_level'] = SUBJECT['education_level'].replace({'Unknown': NULL })

# map "Unknown" to NULL

# map ['14', '12', '16', '18', '13', '8', '15', '20', '19', '17', '11', '22', '10', '21'] ???

In [53]:
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE_df)
report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 94/94 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hx_melanoma*_:  invalid values 💩'Unknown'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>', '14', '12', '16', '18', '13', '8', '15', '20', '19', '17', '11', '22', '10', '21'
    - valid ➡️ 'High School', 'High School/GED', 'Some college without degree', 'Associate degree college', 'Bachelor's degree', 'Master's degree', 'Professional or doctoral degree', 'Refuse', 'Other'
- _*cognitive_status*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Normal', 'MCI', 'Dementia'



In [54]:
SAMPLE.head()

,sample_id,source_sample_id,cellranger_outs_folder,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,PD_BN0009_MTG_snRNAseq_rep1,BN0009,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,BN0009,rep1,1,0,batch1,Brain,Middle temporal gyrus,...,<NA>,<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
1,PD_BN0329_MTG_snRNAseq_rep1,BN0329,/data/neurogen/ASAP/scRNAseq/data/batch2_ASAP_...,BN0329,rep1,1,0,batch2,Brain,Middle temporal gyrus,...,<NA>,<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
2,HC_BN0339_MTG_snRNAseq_rep1,BN0339,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,BN0339,rep1,1,0,batch1,Brain,Middle temporal gyrus,...,<NA>,<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
3,HC_BN0341_MTG_snRNAseq_rep1,BN0341,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,BN0341,rep1,1,0,batch1,Brain,Middle temporal gyrus,...,<NA>,<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
4,HC_BN0347_MTG_snRNAseq_rep1,BN0347,/data/neurogen/ASAP/scRNAseq/data/batch2_ASAP_...,BN0347,rep1,1,0,batch2,Brain,Middle temporal gyrus,...,<NA>,<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>


In [55]:
SAMPLE = prep_table(SAMPLE, CDE_df)

SAMPLE['region_level_1'] = SAMPLE['region_level_1'].replace({'Temporal Lobe':'Temporal lobe'})
SAMPLE['molecular_source'] = SAMPLE['molecular_source'].replace({'PolyA_RNA':'PolyA RNA'})


In [56]:
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"
subj_sex_mapper = dict(zip(SUBJECT['subject_id'], SUBJECT['sex']))
SAMPLE['sex_ontology_term_id'] = SAMPLE['subject_id'].map(subj_sex_mapper).replace({"Male":"PATO:0000384 (male)", "Female":"PATO:0000383 (female)" })


## ERROR in metadata

'source_sample_id' cannot be the same as 'subject_id'

In [57]:
SAMPLE['source_sample_id'] = NULL

In [58]:
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE_df)

report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()

All required fields are present in *SAMPLE* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- DV200: 97/97 empty rows

	- pm_PH: 97/97 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sequencing_length*_:  invalid values 💩'120'
    - valid ➡️ '25', '50', '100', '150'



In [59]:

CLINPATH = prep_table(CLINPATH, CDE_df)



In [60]:
# autopsy coding is too too idosyncratic.  leave 
# the path_nia_aa_a are coded as path_nia_ri... swap them 



# 'Low', 'Intermediate', 'High', 'None'
CLINPATH['path_nia_ri'] = CLINPATH['path_nia_aa_a'].replace({'Low ADNC': 'Low', 
                                                            'Not AD': 'None', 
                                                            'Intermediate ADNC': 'Intermediate'})
CLINPATH['path_nia_aa_a'] = NULL


# path_thal
CLINPATH['path_thal'].replace({'Phase 0 (A0)':'0', 
                                'Phase 1 (A1)':'1', 
                                'Phase 2 (A1)':'2', 
                                'Phase 3 (A2)':'3', 
                                'Phase 4 (A3)':'4',
                                'Phase 5 (A3)':'5', 
                                'Unknown':NULL, 
                                'Missing/unknown':NULL }, inplace=True)


CLINPATH['path_infarcs'].replace('Unknown', NULL, inplace=True)


# no idea what No, undx, yes maps to.  leave
CLINPATH['TDP43'].replace('Unknown', NULL, inplace=True)

# sn_neuronal_loss has alternate 0-3 coding.  leave

In [61]:
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)




In [62]:
# autopsy coding is too too idosyncratic.  leave 
# the path_nia_aa_a are coded as path_nia_ri... swap them 



# 'Low', 'Intermediate', 'High', 'None'
CLINPATH['path_nia_ri'] = CLINPATH['path_nia_aa_a'].replace({'Low ADNC': 'Low', 
                                                            'Not AD': 'None', 
                                                            'Intermediate ADNC': 'Intermediate'})
CLINPATH['path_nia_aa_a'] = NULL


# path_thal
CLINPATH['path_thal'].replace({'Phase 0 (A0)':'0', 
                                'Phase 1 (A1)':'1', 
                                'Phase 2 (A1)':'2', 
                                'Phase 3 (A2)':'3', 
                                'Phase 4 (A3)':'4',
                                'Phase 5 (A3)':'5', 
                                'Unknown':NULL, 
                                'Missing/unknown':NULL }, inplace=True)


CLINPATH['path_infarcs'].replace('Unknown', NULL, inplace=True)


# no idea what No, undx, yes maps to.  leave
CLINPATH['TDP43'].replace('Unknown', NULL, inplace=True)

# sn_neuronal_loss has alternate 0-3 coding.  leave

In [63]:

mckeith_map = {'l. Olfactory Bulb-Only':"Olfactory bulb only", 'lla. Brainstem Predominant':"Brainstem",
       'llb. Limbic Predominant':"Limbic (transitional)", 'lV. Neocortical':"Neocortical",
       'lll. Brainstem/Limbic':"Amygdala Predominant", '0. No Lewy bodies':"Absent", "Unknown":NULL}


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace(mckeith_map)

# leave path_nia_ri like this for now. not sure how to map "criteria not met" and "Not AD"

# leave amyloid_angiopathy_severity_scale like this for now. not sure how to map 'Cerebral amyloid angiopathy, temporal and occipital lobe','Cerebral amyloid angiopathy, frontal lobe']


In [64]:
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- path_year_death: 2/94 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_autopsy_dx_main*_:  invalid values 💩'PD/Dem', 'Control', 'Control (MCI)', 'PD/ND', '<NA>', 'Control, Non-motoric', 'PDD', 'MCI', 'PD (MCI)', 'Control (history)', 'Control (diseased)', 'MCI (clinical history)', 'Control (clinical history)'
    - valid ➡️ 'Lewy body disease nos', 'Parkinson's disease', 'Parkinson's disease with dementia', 'Dementia with Lewy bodies', 'Multiple system atrophy (SND>OPCA)', 'Multiple system atrophy (OPCA<SND)', 'Multiple system atrophy (SND=OPCA)', 'Progressive supranuclear palsy', 'Corticobasal degeneration', 'Globular glial tauoapathy (GGT)', 'Chronic traumatic encephalopathy (CTE)', 'FTLD-Tau (Pick's)', 'FTLD-Tau (MAPT)', 'FTLD-Tau (AGD)', 'FTLD-TDP43, Type A', 'FTLD-TDP43, Type B', 'FTLD-TDP43, Type C', 'FTLD-TDP43, Type D'

In [65]:
DATA.head()

,sample_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name_source,file_name,local_path,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,PD_BN0009_MTG_snRNAseq_rep1,BN0009,rep1,1,0,batch1,fastq,BN0009_S2_L001_I1_001.fastq.gz,PD_BN0009_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,Index 1 Lane 1,bc037ea92d5359f3da1ba6da4f974e1c,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
1,PD_BN0009_MTG_snRNAseq_rep1,BN0009,rep1,1,0,batch1,fastq,BN0009_S2_L001_I2_001.fastq.gz,PD_BN0009_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,Index 2 Lane 1,0c8ef844b76212e82428af03d341048b,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
2,PD_BN0009_MTG_snRNAseq_rep1,BN0009,rep1,1,0,batch1,fastq,BN0009_S2_L001_R1_001.fastq.gz,PD_BN0009_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,Read 1 Lane 1,c6a41f29849152df2ba2391dc559149b,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
3,PD_BN0009_MTG_snRNAseq_rep1,BN0009,rep1,1,0,batch1,fastq,BN0009_S2_L001_R2_001.fastq.gz,PD_BN0009_MTG_snRNAseq_rep1_L001_R2_001.fastq.gz,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,Read 2 Lane 1,4d610fdf5a72c8206b2f1505d8fcecd7,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
4,PD_BN0009_MTG_snRNAseq_rep1,BN0009,rep1,1,0,batch1,fastq,BN0009_S2_L002_I1_001.fastq.gz,PD_BN0009_MTG_snRNAseq_rep1_L002_I1_001.fastq.gz,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,Index 1 Lane 2,e34917b3ec01cd09ea8b52c82db1dc83,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>


In [66]:
DATA_ = DATA.copy()
DATA = reorder_table_to_CDE(DATA, "DATA", CDE_df)

report = ReportCollector(destination="")
validate_table(DATA, "DATA", CDE_df, report)
report.print_log()

All required fields are present in *DATA* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- time: 1552/1552 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in DATA. 🥳



In [67]:
# remove 'None' and '<NA>'

CLINPATH = sanitize(CLINPATH)
PROTOCOL = sanitize(PROTOCOL)
SUBJECT = sanitize(SUBJECT)
STUDY = sanitize(STUDY)
DATA = sanitize(DATA)
SAMPLE = sanitize(SAMPLE)


One last scrub of the columns 

In [68]:

# export_root = metadata_path / "clean"
# if not export_root.exists():
#     export_root.mkdir(parents=True, exist_ok=True)


export_root = Path.cwd() / "clean/team-Scherzer"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)

STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")
DATA.to_csv(export_root / "DATA.csv")


In [69]:
metadata_path = export_root

In [70]:
metadata_version = "v2"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 

export_root

SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(metadata_path / "PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
DATA = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)

# SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
# CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
# STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
# PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
# SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")
# DATA = pd.read_csv(f"{metadata_path}/DATA.csv")


read url


In [40]:
metadata_path

PosixPath('/Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer')

In [71]:
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,PD_BN0009_MTG_snRNAseq_rep1,BN0009,<NA>,rep1,1,0,batch1,Brain,Middle temporal gyrus,Left,...,PATO:0000384 (male),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
1,PD_BN0329_MTG_snRNAseq_rep1,BN0329,<NA>,rep1,1,0,batch2,Brain,Middle temporal gyrus,Left,...,PATO:0000384 (male),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
2,HC_BN0339_MTG_snRNAseq_rep1,BN0339,<NA>,rep1,1,0,batch1,Brain,Middle temporal gyrus,Left,...,PATO:0000384 (male),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
3,HC_BN0341_MTG_snRNAseq_rep1,BN0341,<NA>,rep1,1,0,batch1,Brain,Middle temporal gyrus,Left,...,PATO:0000384 (male),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
4,HC_BN0347_MTG_snRNAseq_rep1,BN0347,<NA>,rep1,1,0,batch2,Brain,Middle temporal gyrus,Left,...,PATO:0000383 (female),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>


In [14]:
# 

# ## convert 
# data_path2 = Path.home() / ("Projects/ASAP/meta-clean/clean/team-Jakobsson")
# metadata_path2 = data_path2 / "v2_20231114"

# SUBJECT_ = read_meta_table(metadata_path2 / "SUBJECT.csv", dtypes_dict)
# CLINPATH_ = read_meta_table(metadata_path2 / "CLINPATH.csv", dtypes_dict)
# STUDY_ = read_meta_table(metadata_path2 / "STUDY.csv", dtypes_dict)
# PROTOCOL_ = read_meta_table(metadata_path2 / "PROTOCOL.csv", dtypes_dict)
# SAMPLE_ = read_meta_table(metadata_path2 / "SAMPLE.csv", dtypes_dict)
# DATA_ = read_meta_table(metadata_path2 / "DATA.csv", dtypes_dict)

In [26]:


# # fix SAMPLE replicate, replicate_count, batch with SAMPLE_

# batch_mapper = dict(zip(DATA['sample_id'], DATA['batch']))
                        
# batch = "batch_" + SAMPLE['sample_id'].map(batch_mapper)
# batch

# SAMPLE['batch']  = batch
# SAMPLE['replicate'] = 'rep1'
# SAMPLE['replicate_count'] = 1
# SAMPLE['repeated_sample'] = 0

# # also fix "replicate" coding in DATA "rep1"
# DATA['replicate'] = 'rep1'

In [41]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz,Index 1 Lane 1,bc037ea92d5359f3da1ba6da4f974e1c,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
1,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz,Index 2 Lane 1,0c8ef844b76212e82428af03d341048b,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
2,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz,Read 1 Lane 1,c6a41f29849152df2ba2391dc559149b,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
3,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_R2_001.fastq.gz,Read 2 Lane 1,4d610fdf5a72c8206b2f1505d8fcecd7,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
4,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L002_I1_001.fastq.gz,Index 1 Lane 2,e34917b3ec01cd09ea8b52c82db1dc83,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>


In [43]:
# checksum = DATA[['file_name','file_MD5']]
# checksum['check'] = checksum['file_name'].map(bucket_files_md5)
# checksum

In [44]:

from utils.checksums import extract_md5_from_details, extract_md5_from_details2,  extract_crc32c_from_details2, extract_hashes_from_gcloudstorage

In [45]:
source_hash = Path.home() / "Projects/ASAP/team-scherzer/scherzer_hash.log"

source_file_crc = extract_crc32c_from_details2(source_hash)
source_file_md5 = extract_md5_from_details2(source_hash)

source_hash.exists()

True

In [46]:
source_file_crc,source_file_md5 = extract_hashes_from_gcloudstorage(source_hash)

## crc32c hashes 

TODO: write hash tools for checking...

In [47]:
bucket_files_md5 = extract_md5_from_details2("scherzer_hexhash.log")


In [48]:
bucket_files_crc = extract_crc32c_from_details2("scherzer_hexhash.log")


In [49]:

checksum = DATA[['file_name','file_MD5']]


checksum['check_src_crc'] = checksum['file_name'].map(source_file_crc)
checksum['check_bucket_crc'] = checksum['file_name'].map(bucket_files_crc)


checksum['meta_MD5'] = checksum['file_MD5'].str.strip()
checksum['check_src_md5'] = checksum['file_name'].map(source_file_md5)
checksum['check_bucket_md5'] = checksum['file_name'].map(bucket_files_md5)




/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_16885/3456434893.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check_src_crc'] = checksum['file_name'].map(source_file_crc)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_16885/3456434893.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check_bucket_crc'] = checksum['file_name'].map(bucket_files_crc)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_16885/3456434893.py:8: SettingWithCopyWarning: 
A v

In [50]:
checksum

,file_name,file_MD5,check_src_crc,check_bucket_crc,meta_MD5,check_src_md5,check_bucket_md5
0,PD_BN0009_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz,bc037ea92d5359f3da1ba6da4f974e1c,c99915b4,c99915b4,bc037ea92d5359f3da1ba6da4f974e1c,bc037ea92d5359f3da1ba6da4f974e1c,NaN
1,PD_BN0009_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz,0c8ef844b76212e82428af03d341048b,f0af5e35,f0af5e35,0c8ef844b76212e82428af03d341048b,0c8ef844b76212e82428af03d341048b,NaN
2,PD_BN0009_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz,c6a41f29849152df2ba2391dc559149b,151d9d97,151d9d97,c6a41f29849152df2ba2391dc559149b,c6a41f29849152df2ba2391dc559149b,NaN
3,PD_BN0009_MTG_snRNAseq_rep1_L001_R2_001.fastq.gz,4d610fdf5a72c8206b2f1505d8fcecd7,58bc244e,58bc244e,4d610fdf5a72c8206b2f1505d8fcecd7,4d610fdf5a72c8206b2f1505d8fcecd7,NaN
4,PD_BN0009_MTG_snRNAseq_rep1_L002_I1_001.fastq.gz,e34917b3ec01cd09ea8b52c82db1dc83,600d0703,600d0703,e34917b3ec01cd09ea8b52c82db1dc83,e34917b3ec01cd09ea8b52c82db1dc83,NaN
...,...,...,...,...,...,...,...
1547,HC_BN1762_MTG_snRNAseq_rep1_L003_R2_001.fastq.gz,887e3d000587b2774ce3cd7022078577,93e1775e,93e1775e,887e3d000587b2774ce3cd7022078577,887e3d000587b2774ce3cd7022078577,NaN
1548,HC_BN1762_MTG_snRNAseq_rep1_L004_I1_001.fastq.gz,44ac0c6d4f7f3b4502eead39564c8723,a361f850,a361f850,44ac0c6d4f7f3b4502eead39564c8723,44ac0c6d4f7f3b4502eead39564c8723,NaN
1549,HC_BN1762_MTG_snRNAseq_rep1_L004_I2_001.fastq.gz,dc114df490e1857072462ca090799005,b617330f,b617330f,dc114df490e1857072462ca090799005,dc114df490e1857072462ca090799005,NaN
1550,HC_BN1762_MTG_snRNAseq_rep1_L004_R1_001.fastq.gz,e24eddb1ac0b1f4d2bdd32b5f69dae27,e49ff1eb,e49ff1eb,e24eddb1ac0b1f4d2bdd32b5f69dae27,e24eddb1ac0b1f4d2bdd32b5f69dae27,NaN


transfer clean metadata


In [1]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/scherzer-credentials.json  



Activated service account credentials for: [raw-admin-scherzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



To take a quick anonymous survey, run:
  $ gcloud survey



In [63]:
!gcloud storage ls -l "gs://asap-raw-data-team-scherzer/metadata/"  --billing-project dnastack-asap-parkinsons 

                                  gs://asap-raw-data-team-scherzer/metadata/Scherzer_raw_meta11202023/
                                  gs://asap-raw-data-team-scherzer/metadata/v2/
TOTAL: 0 objects, 0 bytes (0B)


In [53]:
!gcloud storage cp -r "/Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer" "gs://asap-raw-data-team-scherzer/metadata"  --billing-project dnastack-asap-parkinsons 

Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/CLINPATH.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/CLINPATH.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/SUBJECT.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/SUBJECT.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/SAMPLE.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/SAMPLE.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/DATA.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/DATA.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/STUDY.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/STUDY.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/PROTOCOL.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/PROTOCOL.csv
  Completed files 6/6 | 351.3kiB/351.3kiB                   

In [73]:
!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Scherzer/*.csv"  "gs://asap-raw-data-team-scherzer/metadata/v2"


Copying file://./clean/team-Scherzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Scherzer/SUBJECT.csv [Content-Type=text/csv]...     
Copying file://./clean/team-Scherzer/SAMPLE.csv [Content-Type=text/csv]...      
Copying file://./clean/team-Scherzer/DATA.csv [Content-Type=text/csv]...        
\ [4 files][343.9 KiB/343.9 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Scherzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Scherzer/PROTOCOL.csv [Content-Type=text/csv]...    
\ [6 files][346.6 KiB/346.6 KiB]                                                
Operation completed over 6 objects/346.6 KiB.                                    


In [70]:



# checksum_path = "scherzer_crc32c_checksums.csv"

# chksum = pd.read_csv(checksum_path)
# chksum.head()




,filename,crc32c_checksum
0,ILB_BN9947_MTG_snRNAseq_rep1_L004_R1_001.fastq.gz,904383552
1,ILB_BN9947_MTG_snRNAseq_rep1_L004_R2_001.fastq.gz,3515453253
2,HC_BN0855_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz,4034171979
3,HC_BN0855_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz,1461765174
4,HC_BN0855_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz,2768035529


### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [1]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [66]:
!gsutil cp "./clean/team-Scherzer/*.csv" "gs://asap-workflow-dev/metadata/v2/scherzer"

Copying file://./clean/team-Scherzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Scherzer/SUBJECT.csv [Content-Type=text/csv]...     
Copying file://./clean/team-Scherzer/SAMPLE.csv [Content-Type=text/csv]...      
Copying file://./clean/team-Scherzer/DATA.csv [Content-Type=text/csv]...        
- [4 files][348.6 KiB/348.6 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Scherzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Scherzer/PROTOCOL.csv [Content-Type=text/csv]...    
\ [6 files][351.3 KiB/351.3 KiB]                                                
Operation completed over 6 objects/351.3 KiB.                                    


In [7]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2/lee"

      6243  2023-11-30T21:20:08Z  gs://asap-workflow-dev/metadata/v2/lee/CLINPATH.csv#1701379208316153  metageneration=1
     24214  2023-11-30T21:20:08Z  gs://asap-workflow-dev/metadata/v2/lee/DATA.csv#1701379208935791  metageneration=1
       968  2023-11-30T21:20:09Z  gs://asap-workflow-dev/metadata/v2/lee/PROTOCOL.csv#1701379209349939  metageneration=1
     23092  2023-11-30T21:20:08Z  gs://asap-workflow-dev/metadata/v2/lee/SAMPLE.csv#1701379208725066  metageneration=1
      1054  2023-11-30T21:20:09Z  gs://asap-workflow-dev/metadata/v2/lee/STUDY.csv#1701379209140495  metageneration=1
      4277  2023-11-30T21:20:08Z  gs://asap-workflow-dev/metadata/v2/lee/SUBJECT.csv#1701379208528070  metageneration=1
                                 gs://asap-workflow-dev/metadata/v2/lee/v2_20231128/
TOTAL: 6 objects, 59848 bytes (58.45 KiB)


In [3]:
!gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-scherzer/"

gs://asap-raw-data-team-scherzer/andy_testfile.txt
gs://asap-raw-data-team-scherzer/andy_testfile2.txt
gs://asap-raw-data-team-scherzer/test-raw-scherzer.txt
gs://asap-raw-data-team-scherzer/testfile.txt
gs://asap-raw-data-team-scherzer/testfile_symlink.txt
gs://asap-raw-data-team-scherzer/fastqs/
gs://asap-raw-data-team-scherzer/metadata/
gs://asap-raw-data-team-scherzer/test_dir/
gs://asap-raw-data-team-scherzer/test_gcloud/
gs://asap-raw-data-team-scherzer/test_gcloud2/
gs://asap-raw-data-team-scherzer/test_gcloud3/
gs://asap-raw-data-team-scherzer/test_gsutil/
gs://asap-raw-data-team-scherzer/workflow_execution/


In [4]:
!gsutil -u dnastack-asap-parkinsons du -h "gs://asap-raw-data-team-scherzer/"

0 B          gs://asap-raw-data-team-scherzer/andy_testfile.txt
0 B          gs://asap-raw-data-team-scherzer/andy_testfile2.txt
20 B         gs://asap-raw-data-team-scherzer/test-raw-scherzer.txt
0 B          gs://asap-raw-data-team-scherzer/testfile.txt
15 B         gs://asap-raw-data-team-scherzer/testfile_symlink.txt
225.04 MiB   gs://asap-raw-data-team-scherzer/fastqs/HC_BN0339_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz
228.87 MiB   gs://asap-raw-data-team-scherzer/fastqs/HC_BN0339_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz
725.95 MiB   gs://asap-raw-data-team-scherzer/fastqs/HC_BN0339_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz
2.18 GiB     gs://asap-raw-data-team-scherzer/fastqs/HC_BN0339_MTG_snRNAseq_rep1_L001_R2_001.fastq.gz
223.42 MiB   gs://asap-raw-data-team-scherzer/fastqs/HC_BN0339_MTG_snRNAseq_rep1_L002_I1_001.fastq.gz
224.63 MiB   gs://asap-raw-data-team-scherzer/fastqs/HC_BN0339_MTG_snRNAseq_rep1_L002_I2_001.fastq.gz
710.75 MiB   gs://asap-raw-data-team-scherzer/fastqs/HC_BN0339_MT